In [1]:
import pandas as pd
import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
data=pd.read_csv('/content/drive/MyDrive/iot-dataset/device_5_dataset.csv')

In [4]:
data.head()
data.groupby('type')['type'].count()

type
benign             62154
gafgyt_combo       61380
gafgyt_junk        30898
gafgyt_scan        29297
gafgyt_tcp        104510
gafgyt_udp        104011
mirai_ack          60554
mirai_scan         96781
mirai_syn          65746
mirai_udp         156248
mirai_udpplain     56681
Name: type, dtype: int64

In [5]:
data["type"].replace({"gafgyt_combo": "attack" ,
                             "gafgyt_junk" : "attack",
                             "gafgyt_scan" : "attack",
                             "gafgyt_tcp" : "attack",
                             "gafgyt_udp" : "attack",
                             "mirai_ack" : "attack",
                             "mirai_scan" : "attack",
                             "mirai_syn" : "attack",
                             "mirai_udp" : "attack",
                             "mirai_udpplain" : "attack"},
                             inplace=True)

In [6]:
x = data.iloc[:,:-1].values
y = data.iloc[:,-1].values

In [ ]:
#standardize numerical columns
# def standardize(df,col):
#     df[col]= (df[col]-df[col].mean())/df[col].std()
# data_st=data.copy()
# for i in (data_st.iloc[:,:-1].columns):
#     standardize (data_st,i)
# data_st.head()

In [ ]:
#data["CLASS_LABEL"].replace({"one": 1 , "zero" : 0}, inplace=True)

In [7]:
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
y = labelencoder.fit_transform(y)

In [8]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
x = sc.fit_transform(x)

In [9]:
x = x.reshape(x.shape[0], x.shape[1], 1)
print(x.shape)

(828260, 115, 1)


In [10]:
y

array([1, 1, 1, ..., 0, 0, 0])

In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2,shuffle=True)

In [12]:
from keras import Sequential
from keras.layers import Dense, Conv1D, Flatten, LSTM


In [13]:
#Create model
model = Sequential()
model.add(LSTM(40, activation='relu',input_shape=(115,1,)))    # the batch size is neglected!



model.add(Dense(2, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 40)                6720      
                                                                 
 dense (Dense)               (None, 2)                 82        
                                                                 
Total params: 6,802
Trainable params: 6,802
Non-trainable params: 0
_________________________________________________________________


In [14]:
#Fitting the data to the training dataset
model.fit(X_train,y_train, validation_data=(X_test,y_test), epochs=1)

20707/20707 [==============================] - 1081s 52ms/step - loss: 69512096.0000 - accuracy: 0.9312 - val_loss: 25.5069 - val_accuracy: 0.9295


In [15]:
y_pred=model.predict(X_test)
y_pred = y_pred.argmax(axis=-1)

In [16]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, matthews_corrcoef, f1_score, precision_score, recall_score

print(f"Accuracy   : {accuracy_score(y_test, y_pred)}")
print(f"Precision  : {precision_score(y_test, y_pred)}")
print(f"F1-Measure : {f1_score(y_test, y_pred)}")
print(f"MCC        : {matthews_corrcoef(y_test, y_pred)}")
print(f"Recall     : {recall_score(y_test, y_pred)}")

Accuracy   : 0.9294786661193345
Precision  : 1.0
F1-Measure : 0.12402519496100779
MCC        : 0.24784404674566746
Recall     : 0.06611239907266768


In [17]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.93      1.00      0.96    153143
           1       1.00      0.07      0.12     12509

    accuracy                           0.93    165652
   macro avg       0.96      0.53      0.54    165652
weighted avg       0.93      0.93      0.90    165652



In [18]:
print(confusion_matrix(y_test, y_pred))

[[153143      0]
 [ 11682    827]]
